In [16]:
from bs4 import BeautifulSoup 
from urllib.request import Request, urlopen
import pandas as pd
import json
import time
import re

# PARTIE 1 : COLLECTE DE DONNEES SUR SCIENCEDIRECT

## Comment utiliser le script ? 

### 1) Faire une requête sur Springer avec votre clé de recherche

### 2) Copier url de la première page de recherche dans first_url 

### 3) Lancer le script de démarrage de collecte
Lors de l'excécution du script, le nombre de page et d'article s'affichent. 

Le chargement de données est indiqué pour chaque article. 

Les données collectées sont : 'article', 'conference paper', 'conference paper and chapter'

In [17]:
# lien de la première page de recherche 
first_url='https://rd.springer.com/search/page/1?date-facet-mode=between&showAll=true&query=multi-label*+AND+OR+AND+multilabel*+AND+AND+AND+text*+AND+OR+AND+document+AND+OR+AND+%E2%80%9Cnatural+AND+language+AND+process*%E2%80%9D+AND+AND+AND+class*+AND+OR+AND+categorization*+AND+OR+AND+tag*'

In [18]:
# Collecte le code source d'une page web d'un url donné en argument 
def collecte_source_code(url : str) -> str :
    try : 
        req = Request(url,headers={'User-Agent': 'Chrome/71.0.3578.98'})
        webpage = urlopen(req).read()
        source_code = BeautifulSoup(webpage,'html.parser') 
        return source_code
    except Exception as e :
        print('erreur de chargement de la page... opération recommencée') 
        collecte_source_code(url)

# Cette fonction compte le nombre de page de résultat pour le premier lien de recherche
def page_counter(url : str) -> list :
    soup = collecte_source_code(url)
    try :
        number_page  = int(soup.find('span', { "class" : "number-of-pages"}).getText())
    except : 
        number_page = int(1)
    return number_page

# collecte une liste de titre, date, type de publication, mots clé pour chaque article 
# et crée une dataframe avec ces données
def collect_data(article_list : list) :
    i = 1
    title_list = []
    date_list = []
    keywords_list = []
    publication_type_list = []
    for article in article_list : 
        print("chargement des données de l'article", i)
        soup = collecte_source_code(article)
        try :
            title = soup.find('meta', {'name':'dc.title'}).get('content') # title
            title_list.append(title)
        except : 
            try :
                title = soup.find('meta', {'name':'citation_title'}).get('content') # title
                title_list.append(title)
            except :
                title=0
                title_list.append(title)
        try : 
            publication_date = soup.find('meta', {'name':'dc.date'}).get('content') # date
            date_list.append(publication_date) 
        except :
            try :
                publication_date = soup.find('meta', {'name':'citation_date'}).get('content') # title
                date_list.append(publication_date)
            except :
                publication_date = soup.find('meta', {'name':'citation_publication_date'}).get('content') # title
                date_list.append(publication_date)
        try : 
            script = soup.find('script', attrs={'type':'text/javascript'}).text
            script = script.replace('window.dataLayer = ','')
            script = script.replace(';','')
            json1_data = json.loads(script)[0] # transformation du script json en dictionnaire 
            keywords = json1_data['Keywords'] # key words
            keywords_list.append(keywords)
        except :
            try :
                keywords = soup.findAll("span",{'class':"Keyword"})
                keywords = str(keywords)
                keywords = keywords.replace('<span class="Keyword">','')
                keywords = keywords.replace('</span>','')
                keywords = keywords.replace('[','')
                keywords = keywords.replace(']','')
                keywords_list.append(keywords)
            except : 
                keywords=0
                keywords_list.append(keywords)
        try :     
            publication_type = soup.find('meta', {'name':'dc.type'}).get('content') # publication type
            publication_type_list.append(publication_type)
        except : 
            try : 
                publication_type = soup.find('meta', {'name':'citation_conference_title'}).get('content') # title
                publication_type = 'Conference Paper (Or Conference Paper And Chapter)' 
                publication_type_list.append(publication_type)  
            except : 
                publication_type = 'Chapter'
                publication_type_list.append(publication_type)  
        i=i+1
    create_dataframe(title_list, date_list, keywords_list, publication_type_list)

# Création d'une dataframe à partir des listes de titre, date, type de publication, mots clé pour chaque article
def create_dataframe(title_list : list, date_list : list, keywords_list : list, publication_type_list : list) :
       # crée un dictionnaire à partir des listes
        dict = {'Title': title_list, 'Date': date_list , 'Author Keywords': keywords_list, 'Publication type' : publication_type_list}
        # utilisation du dictionnaraire pour créer une dataframe
        df = pd.DataFrame(dict)
        df = df[df['Publication type'] != 'Chapter']
        df.to_csv('data/dataSPRINGER.csv')


# collecte les liens des pages de recherche et des articles et en extrait les données 
def scrapping_springer(url_list : list) :
    i = 0
    article_list = []
    url_list_article = []
    for url in url_list :
        soup = collecte_source_code(url)
        alllink = soup.find_all(class_= 'title') # liens des articles
        i+=1
        print('collecte des liens des articles sur la page', i)
        for link in alllink :
            article_list.append('https://rd.springer.com/' + link['href']) # tout les liens des articles           
    print("le nombre total d'article est de", len(article_list))
    collect_data(article_list)

# Cette fonction compte le nombre de page de résultat de recherche pour la clé de recherche 
def page_counter(url : str) -> int :
    soup = collecte_source_code(url)
    try :
        number_page  = int(soup.find('span', { "class" : "number-of-pages"}).getText())
    except : 
        number_page = int(1)
    return number_page

# Cette fonction récupère les liens de chaque page de résultat
def collect_all_url_research(url : str) -> list :
    url_list = collect_all_url_research(first_url) # récupère les liens de chaque page
    url_list=[url]
    number_page = page_counter(url)
    if number_page > 1 :
        second_part_url = (url).replace('https://rd.springer.com/search?','')
        second_part_url = second_part_url.replace('*&date-facet-mode=between&showAll=true','')
        for i in range(2, number_page + 1, 1):
            i = str(i)
            new_url = 'https://rd.springer.com/search/page/' + i + '?' + second_part_url
            url_list.append(new_url)
    return url_list # collecte toute les données 

In [19]:
scrapping_springer(url_list) 

collecte des liens des articles sur la page 1
collecte des liens des articles sur la page 2
le nombre total d'article est de 40
chargement des données de l'article 1
chargement des données de l'article 2
chargement des données de l'article 3
chargement des données de l'article 4
chargement des données de l'article 5
chargement des données de l'article 6
chargement des données de l'article 7
chargement des données de l'article 8
chargement des données de l'article 9
chargement des données de l'article 10
chargement des données de l'article 11
chargement des données de l'article 12
chargement des données de l'article 13
chargement des données de l'article 14
chargement des données de l'article 15
chargement des données de l'article 16
chargement des données de l'article 17
chargement des données de l'article 18
chargement des données de l'article 19
chargement des données de l'article 20
chargement des données de l'article 21
chargement des données de l'article 22
chargement des données 